Как зовут водителя (first_name), который совершил наибольшее количество доставок одному клиенту?

In [ ]:
SELECT d.first_name, s.cust_id, COUNT(*) as delivery_count
FROM sql.shipment s
JOIN sql.driver d ON s.driver_id = d.driver_id
GROUP BY d.first_name, s.cust_id
ORDER BY delivery_count DESC
LIMIT 1

Укажите имя клиента, получившего наибольшее количество доставок за 2017 год

In [ ]:
SELECT 
    cust_name, 
    COUNT(ship_id) AS shipment_count
FROM 
    sql.shipment s 
JOIN 
    sql.customer c ON s.cust_id = c.cust_id
WHERE 
    ship_date BETWEEN '2017-01-01' AND '2017-12-31'
GROUP BY 
    cust_name
ORDER BY 
    shipment_count DESC

Допустим, мы хотим собрать из справочников по книгам и фильмам один, так чтобы в нём содержались названия произведений, а также их описание — книга или фильм.

Для этого напишем простой запрос:

In [ ]:
SELECT          book_name object_name, 'книга' object_description 
FROM          public.books
UNION ALL
SELECT          movie_title, 'фильм' 
FROM          sql.kinopoisk

UNION - Сделает уникальные значения  
UNION ALL - Сделает все значения

Важно! UNION оставляет только уникальные значения, а потому требует дополнительных вычислительных мощностей и памяти (в данном случае можно провести аналогию с DISTINCT). Поэтому если вы уверены в отсутствии дубликатов в данных или они вам не важны, предпочтительнее использовать UNION ALL.

Мы уже знаем, что можно легко и непринуждённо применить операторы ORDER BY и LIMIT ко всему результату запроса.

In [ ]:
SELECT book_name object_name, 'книга' object_descritption 
FROM public.books
UNION ALL
SELECT movie_title, 'фильм' 
FROM sql.kinopoisk
ORDER BY 1
LIMIT 1

А если мы не хотим общую сортировку? Может, нам нужны строки с названием как фильма, так и книги, идущих первыми по алфавиту.

Просто добавим ORDER BY и LIMIT ещё и в первую часть запроса:

In [ ]:
(SELECT book_name object_name, 'книга' object_descritption 
FROM public.books
ORDER BY 1
LIMIT 1)
UNION ALL
(SELECT movie_title, 'фильм' 
FROM sql.kinopoisk
ORDER BY 1
LIMIT 1)

Напишите запрос, который создает уникальный алфавитный справочник всех городов, штатов, имён водителей и производителей грузовиков. Результатом запроса должны быть два столбца: название и тип объекта (city, state, driver, truck). Отсортируйте список по названию объекта, а затем — по типу.

In [ ]:
SELECT
  city_name as object_name,
  'city' as object_description 
FROM sql.city

UNION
SELECT state, 'state' 
FROM sql.city

UNION
SELECT first_name, 'driver' 
FROM sql.driver

UNION
SELECT make, 'truck' 
FROM sql.truck

ORDER BY 1, 2

Напишите запрос, который соберёт имена всех упомянутых городов и штатов из таблицы city. Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

In [ ]:
SELECT city_name as object_name
FROM sql.city

UNION ALL
SELECT state
FROM sql.city

ORDER BY object_name

Для типизации в Postgres составляется запрос по модели column_name::column_type

In [ ]:
SELECT 
         c.city_id::text /*выбираем столбец city_id, переводим city_id из числового в текстовый формат*/
FROM
         sql.city c /*из схемы sql  и таблицы city, задаём таблице алиас с*/

UNION ALL /*оператор присоединения*/

SELECT 
         cc.city_name /*выбираем столбец city_name*/
FROM
         sql.city cc /*из схемы sql и таблицы city, задаём таблице алиас сс*/

Напишите запрос, который объединит в себе все почтовые индексы водителей и их телефоны в единый столбец-справочник contact. Также добавьте столбец с именем водителя first_name и столбец contact_type с типом контакта (phone или zip в зависимости от типа). Отсортируйте список по столбцу с контактными данными в порядке возрастания, а затем — по имени водителя.

In [ ]:
SELECT
    zip_code::text AS contact,
    first_name,
    'zip' AS contact_type
FROM sql.driver

UNION ALL

SELECT
    phone,
    first_name,
    'phone'
FROM sql.driver

ORDER BY 
    contact, first_name

Попробуем вывести обобщённые данные о населении по всем городам, с детализацией до конкретного города.

In [ ]:
SELECT
         c.city_name,
         c.population
FROM
         sql.city c
UNION ALL
SELECT
         'total',
         SUM(c.population)
FROM
         sql.city c
ORDER BY 2 DESC

Помимо соединения разнородных сущностей в единый справочник, UNION ALL часто используется для подведения промежуточных итогов и выведения результатов агрегатных функций.

In [ ]:
SELECT
         c.city_name,
         c.population
FROM
         sql.city c
UNION ALL
SELECT
         'total',
         SUM(c.population)
FROM
         sql.city c
ORDER BY 2 DESC

Напишите запрос, который выводит общее число доставок total_shipments, а также количество доставок в каждый день. Необходимые столбцы: date_period, cnt_shipment. Не забывайте о единой типизации. Упорядочите по убыванию столбца date_period.

In [ ]:
(SELECT ship_date::text date_period, COUNT(ship_id) cnt_shipment
FROM sql.shipment
GROUP BY ship_date
ORDER BY ship_date desc)

UNION ALL

SELECT 'total_shipments', COUNT(ship_id)
FROM sql.shipment

UNION также может быть использован для разделения существующей выборки по критерию «выполнение определённого условия».
с помощью UNION можно отобразить, у кого из водителей заполнен столбец с номером телефона.

In [ ]:
SELECT
         d.first_name,
         d.last_name,
         'телефон заполнен' phone_info
FROM
         sql.driver d
WHERE d.phone IS NOT NULL

UNION

SELECT
         d.first_name,
         d.last_name,
         'телефон не заполнен' phone_info
FROM
         sql.driver d
WHERE d.phone IS NULL

In [ ]:
SELECT
    city_name AS city_name,
    state AS state,
    'доставка осуществлялась' AS shipping_status
FROM sql.city c 
LEFT JOIN sql.shipment s on c.city_id = s.city_id
WHERE cust_id is not null

union

SELECT
    city_name AS city_name,
    state AS state,
    'доставка не осуществлялась' AS shipping_status
FROM sql.city c 
LEFT JOIN sql.shipment s on c.city_id = s.city_id
WHERE cust_id is null

ORDER BY city_name, state

Напишите запрос, который выводит два столбца: city_name и shippings_fake. Выведите города, куда совершались доставки. Пусть первый столбец содержит название города, а второй формируется так:

- если в городе было более десяти доставок, вывести количество доставок в этот город как есть;
- иначе — вывести количество доставок, увеличенное на пять.

Отсортируйте по убыванию получившегося «нечестного» количества доставок, а затем — по имени в алфавитном порядке.

In [ ]:
SELECT
    city_name AS city_name,
    COUNT(cust_id) AS shippings_fake
FROM sql.city c 
JOIN sql.shipment s on c.city_id = s.city_id
GROUP BY city_name
HAVING COUNT(cust_id) > 10

UNION

SELECT
    city_name AS city_name,
    COUNT(cust_id) + 5 AS shippings_fake
FROM sql.city c 
JOIN sql.shipment s on c.city_id = s.city_id
GROUP BY city_name
HAVING COUNT(cust_id) <= 10

ORDER BY shippings_fake desc, city_name

UNION можно использовать для создания справочников прямо в коде запроса. К примеру, если мы хотим вручную ввести какие-то значения и произвести с ними некоторые манипуляции или дополнить существующую выдачу своими значениями.

In [ ]:
SELECT 
         'a' letter,'1' ordinal_position
UNION 
SELECT 
         'b','2'
UNION 
SELECT
         'c','3'

Существуют сложные алгоритмы сравнения текстовых значений, но главный смысл сводится к одному: сравнение производится на основе таблицы unicode и позиции элемента в ней с учётом определённых условий.

Напишите запрос, который выберет наибольшее из числовых значений:

- 1000000;
- 541;
- -500;
- 100.
Столбец с результатом назовите result.

In [ ]:
SELECT 
        1000000 AS result
UNION 
SELECT 
        541
UNION 
SELECT
        -500
UNION
SELECT 
        100
ORDER BY result desc
LIMIT 1

Помним, что сортировка для числовых и строковых типов данных отличается. Построив запрос по аналогии с примером, приведите значения к текстовому типу данных, сравните и выберите наибольшее из них:

- 1000000;
- 541;
- -500;
- 100.
Столбец с ответом назовите mycol

In [ ]:
SELECT 
        '1000000' AS mycol
UNION 
SELECT 
        '541'
UNION 
SELECT
        '-500'
UNION
SELECT 
        '100'
ORDER BY mycol desc
limit 1

In [ ]:
SELECT 
        '+' AS result
UNION 
SELECT 
        '-'
UNION 
SELECT
        '='
UNION
SELECT 
        '/'
ORDER BY result desc
limit 1

Предположим, нам нужно узнать, в какие города осуществлялась доставка, за исключением тех, в которых проживают водители

In [ ]:
SELECT
         c.city_name
FROM
         sql.shipment s
JOIN sql.city c ON s.city_id = c.city_id
EXCEPT
SELECT
         cc.city_name
FROM
         sql.driver d 
JOIN sql.city cc ON d.city_id=cc.city_id
ORDER BY 1

Все водители проживают в городе Memphis, и мы видим, что он не выводится в результате запроса.

Как вы, должно быть, заметили, для решения этой задачи мы использовали оператор EXCEPT.

Мы уже знаем, как решить такую задачу с использованием LEFT JOIN. Вариант с EXCEPT будет полезен в тех случаях, когда у вас много столбцов и вам не хочется прописывать их равенство в условии для JOIN.

Выведите список zip-кодов, которые есть в таблице sql.driver, но отсутствуют в таблице sql.customer. Отсортируйте по возрастанию, столбец к выводу — zip. В поле ниже введите запрос, с помощью которого вы решили эту задачу.

In [ ]:
SELECT zip_code
FROM sql.driver

EXCEPT

SELECT zip
FROM sql.customer

Предположим, нам надо вывести совпадающие по названию города и штаты.

In [ ]:
SELECT          c.city_name object_name
FROM          sql.city c
INTERSECT
SELECT 
         cc.state
FROM          sql.city cc
ORDER BY 1

Синтаксис запроса с оператором INTERSECT выглядит следующим образом:

In [ ]:
SELECT 
         n columns
FROM 
         table_1
INTERSECT
SELECT 
         n columns
FROM 
         table_2

Напишите запрос, который выведет список id городов, в которых есть и клиенты, и доставки, и водители.

In [ ]:
SELECT city_id
FROM sql.city
INTERSECT
SELECT cust.city_id
FROM sql.customer cust
INTERSECT
SELECT ship.city_id
FROM sql.shipment ship
INTERSECT
SELECT drive.city_id
FROM sql.driver drive


Выведите zip-код, который есть как в таблице с клиентами, так и в таблице с водителями.

In [ ]:
SELECT zip
FROM sql.customer
INTERSECT
SELECT zip_code
FROM sql.driver

Запишем структуру запроса с учётом полученных знаний.

In [ ]:
SELECT          N columns
FROM          table_1
UNION / UNION ALL / EXCEPT / INTERSECT 
SELECT          N columns
FROM          table_2

Выведите города с максимальным и минимальным весом единичной доставки. Столбцы к выводу — city_name, weight.

In [ ]:
(SELECT city_name, max(weight) AS weight
FROM sql.shipment s
JOIN sql.city c on c.city_id = s.city_id
GROUP BY city_name
ORDER BY weight desc
LIMIT 1)

UNION

(SELECT city_name, min(weight) AS weight
FROM sql.shipment s
JOIN sql.city c on c.city_id = s.city_id
GROUP BY city_name
ORDER BY weight
LIMIT 1)

Выведите идентификационные номера клиентов (cust_id), которые совпадают с идентификационными номерами доставок (ship_id). Столбец к выводу — mutual_id. Отсортируйте по возрастанию.

In [ ]:
SELECT cust_id AS mutual_id
FROM sql.customer

INTERSECT

SELECT ship_id
FROM sql.shipment

ORDER BY mutual_id

Создайте справочник, содержащий уникальные имена клиентов, которые являются производителями (cust_type='manufacturer'), и производителей грузовиков, а также описание объекта — 'КЛИЕНТ' или 'ГРУЗОВИК'. Столбцы к выводу — object_name, object_description. Отсортируйте по названию в алфавитном порядке.

In [ ]:
SELECT
    cust_name AS object_name,
    'КЛИЕНТ' AS object_description
FROM sql.customer
WHERE cust_type='manufacturer'

UNION 

SELECT
    make,
    'ГРУЗОВИК'
FROM sql.truck

ORDER BY object_name